In [7]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import pickle

In [3]:
train_slice_df = pd.read_csv("slice_std_train.csv")
test_slice_df = pd.read_csv("slice_std_test.csv")
print(train_slice_df.shape, test_slice_df.shape)

(42800, 361) (10700, 361)


In [5]:
# Open the pickle file in binary read mode
with open('result.pickle', 'rb') as file:
    old_results  = pickle.load(file)

In [8]:
X_train = train_slice_df.drop('Y', axis=1)
y_train = train_slice_df['Y']
X_test = test_slice_df.drop('Y', axis=1)
y_test = test_slice_df['Y']

## Ridge Model

In [9]:
# 1. Fit ridge regression to the train data for a range of lambda values
lambd_values_ridge = np.logspace(1, 8, 100)

In [10]:
# 2. Fit Ridge regression with built-in cross-validation
ridge_cv = RidgeCV(alphas=lambd_values_ridge, store_cv_values=True)
ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([1.00000000e+01, 1.17681195e+01, 1.38488637e+01, 1.62975083e+01,
       1.91791026e+01, 2.25701972e+01, 2.65608778e+01, 3.12571585e+01,
       3.67837977e+01, 4.32876128e+01, 5.09413801e+01, 5.99484250e+01,
       7.05480231e+01, 8.30217568e+01, 9.77009957e+01, 1.14975700e+02,
       1.35304777e+02, 1.59228279e+02, 1.87381742e+02, 2.20513074e+02,
       2.59502421e+02, 3.05385551e+0...
       2.36448941e+06, 2.78255940e+06, 3.27454916e+06, 3.85352859e+06,
       4.53487851e+06, 5.33669923e+06, 6.28029144e+06, 7.39072203e+06,
       8.69749003e+06, 1.02353102e+07, 1.20450354e+07, 1.41747416e+07,
       1.66810054e+07, 1.96304065e+07, 2.31012970e+07, 2.71858824e+07,
       3.19926714e+07, 3.76493581e+07, 4.43062146e+07, 5.21400829e+07,
       6.13590727e+07, 7.22080902e+07, 8.49753436e+07, 1.00000000e+08]),
        store_cv_values=True)

In [11]:
# 3. Fit the model with the selected lambda
ridge_model = Ridge(alpha=ridge_cv.alpha_)
ridge_model.fit(X_train, y_train)

Ridge(alpha=220.51307399030458)

In [12]:
# 4. Extract the ridge coefficients
ridge_coefficients = ridge_model.coef_

In [13]:
# 5. Predict the test set
y_pred_ridge = ridge_model.predict(X_test)

In [15]:
# Performance Metrics for Ridge
validation_ridge = {
    'AIC': None, # AIC is not typically used in ridge regression
    'R2': r2_score(y_test, y_pred_ridge),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_ridge)),
    'MAE': mean_absolute_error(y_test, y_pred_ridge),
    'num_variables': np.sum(ridge_coefficients != 0)
}
validation_ridge

{'AIC': None,
 'R2': 0.858229094231964,
 'RMSE': 8.420103292960205,
 'MAE': 6.213675468649426,
 'num_variables': 360}

## Lasso Model

In [27]:
# 6. Fit lasso model for a range of hyperparameters
lambd_values_lasso = np.logspace(-2, 2, 100)

In [18]:
# 7. Use cross validation (LassoCV) to find lambda parameter value
lasso_cv = LassoCV(alphas=lambd_values_lasso, cv=5, max_iter=10000, n_jobs=-1)
lasso_cv.fit(X_train, y_train)

LassoCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02,
       1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02,
       2.10490414e-02, 2.31012970e-02, 2.53536449e-02, 2.78255940e-02,
       3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02,
       4.43062146e-02, 4.86260158e-02, 5.33669923e-02, 5.85702082e-02,
       6.42807312e-02, 7.05480231e-0...
       1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01,
       1.70735265e+01, 1.87381742e+01, 2.05651231e+01, 2.25701972e+01,
       2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01,
       3.59381366e+01, 3.94420606e+01, 4.32876128e+01, 4.75081016e+01,
       5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01,
       7.56463328e+01, 8.30217568e+01, 9.11162756e+01, 1.00000000e+02]),
        cv=5, max_iter=10000, n_jobs=-1)

In [19]:
# 8. Fit the model with the selected lambda
lasso_model = Lasso(alpha=lasso_cv.alpha_)
lasso_model.fit(X_train, y_train)

Lasso(alpha=0.01)

In [20]:
# 9. Find lasso coefficients and check if any of them were removed by regularization
lasso_coefficients = lasso_model.coef_

In [21]:
# 10. Predict the test set
y_pred_lasso = lasso_model.predict(X_test)

In [22]:
# Performance Metrics for Lasso
validation_lasso = {
    'AIC': None, # AIC is not typically used in lasso regression
    'R2': r2_score(y_test, y_pred_lasso),
    'RMSE': np.sqrt(mean_squared_error(y_test, y_pred_lasso)),
    'MAE': mean_absolute_error(y_test, y_pred_lasso),
    'num_variables': np.sum(lasso_coefficients != 0)
}
validation_lasso

{'AIC': None,
 'R2': 0.8580481710035675,
 'RMSE': 8.425474305254362,
 'MAE': 6.2043910454752424,
 'num_variables': 325}

## Dump Answer

In [24]:
old_results['Validation']['validation_ridge'] = validation_ridge
old_results['Validation']['validation_lasso'] = validation_lasso
old_results['Validation']

{'validation_reduced_lm': {'AIC': 302514.48178790975,
  'R2': 0.8638882045172942,
  'RMSE': 8.450981711744108,
  'MAE': 6.249744339096156,
  'num_variables': 245},
 'validation_PCR': {'AIC': 302506.1882273548,
  'R2': 0.8638954982621725,
  'RMSE': 8.447360569858315,
  'MAE': 6.245095210025417,
  'num_variables': 242},
 'validation_ridge': {'AIC': None,
  'R2': 0.858229094231964,
  'RMSE': 8.420103292960205,
  'MAE': 6.213675468649426,
  'num_variables': 360},
 'validation_lasso': {'AIC': None,
  'R2': 0.8580481710035675,
  'RMSE': 8.425474305254362,
  'MAE': 6.2043910454752424,
  'num_variables': 325}}

In [25]:
res = {'Validation': old_results['Validation'],
        'Ridge_Model' : ridge_model,
        'Lasso_Model' : lasso_model}

In [26]:
with open('result1.pickle', 'wb') as f:
    pickle.dump(res,f)